In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

data = pd.read_csv("/content/fmnist_small.csv")

data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0.0,7.0,0.0,50.0,205.0,196.0,213.0,165.0,0.0,0.0
1,7,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,1,0,0,0,...,142.0,142.0,142.0,21.0,0.0,3.0,0.0,0.0,0.0,0.0
3,8,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,0,0,0,0,0,0,0,0,0,...,213.0,203.0,174.0,151.0,188.0,10.0,0.0,0.0,0.0,0.0


In [ ]:
X = data.iloc[:,1:].values
y=data.iloc[:,0].values

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=143)


In [ ]:
# scaleing
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
from torch.utils.data import Dataset,DataLoader
class Custom_dataset(Dataset):

  def __init__(self,feature,label):
    self.feature = torch.tensor(feature,dtype=torch.float32).reshape(-1,1,28,28)
    self.label = torch.tensor(label,dtype=torch.long)

  def __len__(self):
    return len(self.feature)

  def __getitem__(self,index):
    return self.feature[index],self.label[index]



In [ ]:
train_data = Custom_dataset(X_train,y_train)
test_data = Custom_dataset(X_test,y_test)

In [ ]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False)

In [ ]:
import torch.nn as nn
class My_model(nn.Module):

  def __init__(self,chanels):
    super().__init__()

    self.convolution_ = nn.Sequential(
        nn.Conv2d(chanels,32,kernel_size=3,padding='same'),
        # The first argument is the number of input_feature,Output_feature
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2),

        nn.Conv2d(32,64,kernel_size=3,padding='same'),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3,stride=2)
    )

    self.classification = nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*6*6,128),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Dropout(p=0.4),
        nn.Linear(64,10)

    )

  def forward(self,x):
    output = self.convolution_(x)
    output = self.classification(output)
    return output



In [ ]:
model = My_model(1)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=1e-4)
epochs=100

In [ ]:
# Training
for epoch in range(epochs):
  total_loss=0
  for batch_feature,batch_label in train_loader:
    y_pred = model(batch_feature)

    loss = loss_func(y_pred,batch_label)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()

  avg_loss = total_loss/len(train_loader)

  print(f'{epoch+1}: loss:{avg_loss}')




1: loss:2.018747601164393
2: loss:1.3840172132813786
3: loss:1.0800450314958412
4: loss:0.9330534561570868
5: loss:0.857026159045208
6: loss:0.7812242227864553
7: loss:0.7256847203495991
8: loss:0.6979599932590163
9: loss:0.6480799629745713
10: loss:0.6231726817337864
11: loss:0.5942845807736179
12: loss:0.5811547475406923
13: loss:0.5593295668263033
14: loss:0.5494201010968311
15: loss:0.5229232824710478
16: loss:0.49377274710729896
17: loss:0.4839675110506724
18: loss:0.46422776238745955
19: loss:0.45491343090333136
20: loss:0.4508274488420372
21: loss:0.424530628933964
22: loss:0.402275487780571
23: loss:0.38831978713173465
24: loss:0.39520970184400855
25: loss:0.38338153171970185
26: loss:0.3631665338593793
27: loss:0.36316692093050623
28: loss:0.3487124149519277
29: loss:0.3385399657200618
30: loss:0.3219452223863946
31: loss:0.3205845860472645
32: loss:0.2972507173397455
33: loss:0.27728151081197233
34: loss:0.2828224852501628
35: loss:0.26589227467775345
36: loss:0.2728912691215

In [ ]:
model.eval()

My_model(
  (convolution_): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classification): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2304, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
# evaluation code

total = 0
correct = 0

with torch.no_grad():
  for batch_features,batch_labels in test_loader:

    outputs=model(batch_features)

    predicted_labels=torch.max(outputs,1)[1]

    total=total+batch_labels.shape[0]

    correct+=(predicted_labels==batch_labels).sum().item()

print(correct/total)


0.8803030303030303


TRANSFER LEARNING

In [ ]:
import torch
from torchvision.transforms import transforms

In [ ]:
custom_tranform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [ ]:
from PIL import Image
import numpy as np

In [ ]:
from torch.utils.data import Dataset,DataLoader
class Customdataset(Dataset):

  def __init__(self,feature,label,transform):
    self.feature = feature
    self.label = label
    self.transform = transform

  def __len__(self):
    return len(self.feature)

  def __getitem__(self,index):

    image = self.feature[index].reshape(28,28)

    image = image.astype(np.uint8)

    image = np.stack([image]*3,axis=-1) # (C,H,W) but we want (H,W,C)

    image = Image.fromarray(image)

    image = self.transform(image)

    return image,torch.tensor(self.label[index],dtype=torch.long)


In [ ]:
ar=[[1,2,4,5],[1,2,3,8]]
i=np.stack([ar]*3)
i

array([[[1, 2, 4, 5],
        [1, 2, 3, 8]],

       [[1, 2, 4, 5],
        [1, 2, 3, 8]],

       [[1, 2, 4, 5],
        [1, 2, 3, 8]]])

In [ ]:
i.shape

(3, 2, 4)

In [ ]:
h=np.stack([ar]*3,axis=-1)
h.shape

(2, 4, 3)

In [ ]:
train_data = Customdataset(X_train,y_train,custom_tranform)
test_data = Customdataset(X_test,y_test,custom_tranform)

In [ ]:
train_loader = DataLoader(train_data,batch_size=32,shuffle=True,pin_memory=True)
test_loader = DataLoader(test_data,batch_size=32,shuffle=False,pin_memory=True)

In [ ]:
# fetech the pretrain model
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 57.0MB/s]


In [ ]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
vgg16.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [ ]:
for params in vgg16.features.parameters():
  params.requires_grad = False

In [ ]:
# We need the change the vgg16 classfier architecture
import torch.nn as nn
vgg16.classifier = nn.Sequential(
    nn.Linear(25088,1024),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(1024,512),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(512,10)
)

In [ ]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
learning_rate = 0.0001
epochs=10

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.classifier.parameters(),lr=learning_rate) # Telling to only update the classifier parameters

In [ ]:
# Training
for epoch in range(epochs):
  total_loss=0
  for batch_feature,batch_label in train_loader:
    y_pred = vgg16(batch_feature)

    loss = loss_func(y_pred,batch_label)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_loss+=loss.item()

  avg_loss = total_loss/len(train_loader)

  print(f'{epoch+1}: loss:{avg_loss}')




1: loss:0.9515061218973616
